In [1]:
import scanpy as sc
import numpy as np
import torch

import os,sys
pwd = '/home/lijiahao/workbench/SAVE/'
sys.path.append(pwd)
os.chdir(pwd)

from model.save_model import SAVE
from model.utils.process_h5ad import batch_scale, preprocessing_rna

In [2]:
# download the data via link: https://figshare.com/ndownloader/files/24539828

In [3]:
adata = sc.read_h5ad('pbmc_vars_sb.h5ad')
adata.obs["cell_type"] = adata.obs["final_annotation"]

In [4]:
adata = preprocessing_rna(adata, is_batch_scale=True)

/mnt/sdc/lijiahao/miniconda3/envs/SAVE/lib/python3.9/site-packages/scanpy/preprocessing/_simple.py:139: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['n_genes'] = number
batch_scale: 100%|████████████████████████████████| 1/1 [00:00<00:00,  2.58it/s]


In [5]:
adata.write_h5ad('processed_pbmc.h5ad')

In [7]:
device = torch.device('cuda:1')
seed = 1202
kwargs = {
    "device": device,
    "seed": seed,
    "is_data_scaled": True,
}
import yaml

with open(pwd + "/model_setting.yml", "r") as f:
    setting = yaml.safe_load(f)

kwargs.update(setting["SAVE-B"]["train"])
kwargs.update(setting["SAVE-B"]["model"])

In [8]:
kwargs['batch_size'] = 32

In [9]:
from model.save_model import SAVE

save_model = SAVE(
    adata=adata.copy(),
    is_initialized=True,
    condition_cols=["batch"],
    **kwargs,
)


total col comb is: 1
initialze model weight with seed 1202


In [10]:
save_model.train(**kwargs)
save_model.save_ckpt('ckpt/pancreas_no_compress.pt')

cond encoding max: 8 min:0
total iter: 102400


  0%|                                                   | 0/200 [01:45<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
save_model.load_ckpt('ckpt/pancreas_no_compress.pt')

In [ ]:
latent = save_model.get_latent()

### scIB evaluation

In [39]:
from scib_metrics.benchmark import Benchmarker

In [40]:
adata.obsm['SAVE'] = latent

In [ ]:
import scib
bm = Benchmarker(
    adata,
    batch_key='batch',
    label_key='celltype',
)